In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [77]:
raw_data=spark.read.csv("closing_odds.csv",header=True,inferSchema=True)
raw_data.show()

+------+--------------------+----------+---------------+------+-----------------+------+---------+-------+---------+---------+-------+---------+----------------+---------------+----------------+-------+-----+-------+----+----+
|    id|              league|match_date|          teamA|scoreA|            teamB|scoreB|avg_win_1|avg_tie|avg_win_2|max_win_1|max_tie|max_win_2|top_bookie_win_1| top_bookie_tie|top_bookie_win_2|n_win_1|n_tie|n_win_2|_c19|_c20|
+------+--------------------+----------+---------------+------+-----------------+------+---------+-------+---------+---------+-------+---------+----------------+---------------+----------------+-------+-----+-------+----+----+
|170088|England: Premier ...|2005-01-01|      Liverpool|     0|          Chelsea|     1|   2.9944| 3.1944|   2.2256|      3.2|   3.25|     2.29|     Paddy Power|    Sportingbet|          Expekt|      9|    9|      9|null|null|
|170089|England: Premier ...|2005-01-01|         Fulham|     3|   Crystal Palace|     1|   1

In [78]:
raw_data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- league: string (nullable = true)
 |-- match_date: string (nullable = true)
 |-- teamA: string (nullable = true)
 |-- scoreA: string (nullable = true)
 |-- teamB: string (nullable = true)
 |-- scoreB: string (nullable = true)
 |-- avg_win_1: string (nullable = true)
 |-- avg_tie: double (nullable = true)
 |-- avg_win_2: double (nullable = true)
 |-- max_win_1: double (nullable = true)
 |-- max_tie: double (nullable = true)
 |-- max_win_2: double (nullable = true)
 |-- top_bookie_win_1: string (nullable = true)
 |-- top_bookie_tie: string (nullable = true)
 |-- top_bookie_win_2: string (nullable = true)
 |-- n_win_1: string (nullable = true)
 |-- n_tie: string (nullable = true)
 |-- n_win_2: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)



In [79]:
print raw_data.count()
raw_data=raw_data.filter(raw_data.n_win_1>3)
print raw_data.count()

479440
428763


In [80]:
data=raw_data.select(raw_data['scoreA'],raw_data['scoreB'],raw_data['avg_win_1'],
                     raw_data['avg_tie'],raw_data['avg_win_2'],raw_data['max_win_1'],
                     raw_data['max_tie'],raw_data['max_win_2'])
data.show()

+------+------+---------+-------+---------+---------+-------+---------+
|scoreA|scoreB|avg_win_1|avg_tie|avg_win_2|max_win_1|max_tie|max_win_2|
+------+------+---------+-------+---------+---------+-------+---------+
|     0|     1|   2.9944| 3.1944|   2.2256|      3.2|   3.25|     2.29|
|     3|     1|   1.9456| 3.2333|   3.6722|     2.04|    3.3|     4.15|
|     1|     0|   1.8522| 3.2611|   4.0144|      2.0|    3.4|      4.5|
|     1|     1|   1.6122| 3.4133|   5.4722|     1.67|   3.57|     6.27|
|     1|     3|   5.9878| 3.4778|   1.5567|      7.0|    3.6|     1.62|
|     2|     1|      1.7| 3.3667|   4.7344|     1.73|    3.5|      5.0|
|     2|     1|   1.8622|   3.25|   3.9622|     1.91|    3.4|     4.33|
|     1|     1|   1.6344| 3.3933|   5.2633|      1.7|   3.54|     6.14|
|     5|     2|   2.2156| 3.1722|   3.0444|     2.29|    3.2|     3.35|
|     0|     2|   3.3611| 3.2222|   2.0489|     3.75|   3.25|      2.1|
|     0|     1|   1.6875| 3.3625|   4.5188|     1.73|    3.5|   

In [81]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
def add_result(scoreA,scoreB):
    if scoreA>scoreB:
        return 0
    elif scoreA==scoreB:
        return 1
    else:
        return 2
udf_add_result=F.udf(add_result,T.IntegerType())
data=data.select("*",udf_add_result(data.scoreA,data.scoreB).alias('result'))
data.show()


+------+------+---------+-------+---------+---------+-------+---------+------+
|scoreA|scoreB|avg_win_1|avg_tie|avg_win_2|max_win_1|max_tie|max_win_2|result|
+------+------+---------+-------+---------+---------+-------+---------+------+
|     0|     1|   2.9944| 3.1944|   2.2256|      3.2|   3.25|     2.29|     2|
|     3|     1|   1.9456| 3.2333|   3.6722|     2.04|    3.3|     4.15|     0|
|     1|     0|   1.8522| 3.2611|   4.0144|      2.0|    3.4|      4.5|     0|
|     1|     1|   1.6122| 3.4133|   5.4722|     1.67|   3.57|     6.27|     1|
|     1|     3|   5.9878| 3.4778|   1.5567|      7.0|    3.6|     1.62|     2|
|     2|     1|      1.7| 3.3667|   4.7344|     1.73|    3.5|      5.0|     0|
|     2|     1|   1.8622|   3.25|   3.9622|     1.91|    3.4|     4.33|     0|
|     1|     1|   1.6344| 3.3933|   5.2633|      1.7|   3.54|     6.14|     1|
|     5|     2|   2.2156| 3.1722|   3.0444|     2.29|    3.2|     3.35|     0|
|     0|     2|   3.3611| 3.2222|   2.0489|     3.75

In [84]:
data=data.withColumn("pc",1/data.avg_win_1)
data=data.withColumn("f1",1/data.avg_tie)
data=data.withColumn("f2",1/data.avg_win_2)
data=data.withColumn("f3",1/data.max_win_1)
data=data.withColumn("f4",1/data.max_tie)
data=data.withColumn("f5",1/data.max_win_2)
data.show()

+------+------+---------+-------+---------+---------+-------+---------+------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|scoreA|scoreB|avg_win_1|avg_tie|avg_win_2|max_win_1|max_tie|max_win_2|result|                 pc|                 f1|                 f2|                 f3|                 f4|                 f5|
+------+------+---------+-------+---------+---------+-------+---------+------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|     0|     1|   2.9944| 3.1944|   2.2256|      3.2|   3.25|     2.29|     2|0.33395671920919046|0.31304783370899075|0.44931703810208484|             0.3125| 0.3076923076923077| 0.4366812227074236|
|     3|     1|   1.9456| 3.2333|   3.6722|     2.04|    3.3|     4.15|     0| 0.5139802631578947|  0.309281538984938|  0.272316322640379|0.49019607843137253|0.30303030303030304|0.24096385542168672|
|    

In [8]:
import numpy as np
def compute_p(data,bin_number):
    bins=np.zeros(bin_number+1)
    threshold=100 #the minimum number of games in a legal bin
    p_real=np.zeros((bin_number,2))#for each bin, the first is p of win_1, the second is p of win_2
    for i in range(bin_number+1):
        bins[i]=i*0.0125
    #for each bin calculate p_win_1, p_win_2
    for i in range(bin_number):
        print(i)
        tmp=data.filter(data.pc>bins[i]).filter(data.pc<bins[i+1])
        number_of_bin=tmp.count()
        if number_of_bin<threshold:
            print 'bin number',i,'is',number_of_bin,'not legal'
            continue
        number_win_1=tmp.filter(tmp.result==0).count()
        number_win_2=tmp.filter(tmp.result==2).count()
        #print 'win1 number is',number_win_1
        #print 'bin number is ',number_of_bin
        p_real[i,0]=float(number_win_1)/number_of_bin
        p_real[i,1]=float(number_win_2)/number_of_bin
    return bins,p_real
bins,p_real=compute_p(data,80)

0
bin number 0 is 0 not legal
1
bin number 1 is 32 not legal
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79


In [85]:
print bins
print p_real

[ 0.      0.0125  0.025   0.0375  0.05    0.0625  0.075   0.0875  0.1
  0.1125  0.125   0.1375  0.15    0.1625  0.175   0.1875  0.2     0.2125
  0.225   0.2375  0.25    0.2625  0.275   0.2875  0.3     0.3125  0.325
  0.3375  0.35    0.3625  0.375   0.3875  0.4     0.4125  0.425   0.4375
  0.45    0.4625  0.475   0.4875  0.5     0.5125  0.525   0.5375  0.55
  0.5625  0.575   0.5875  0.6     0.6125  0.625   0.6375  0.65    0.6625
  0.675   0.6875  0.7     0.7125  0.725   0.7375  0.75    0.7625  0.775
  0.7875  0.8     0.8125  0.825   0.8375  0.85    0.8625  0.875   0.8875
  0.9     0.9125  0.925   0.9375  0.95    0.9625  0.975   0.9875  1.    ]
[[ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.97692308]
 [ 0.02340426  0.94468085]
 [ 0.01579779  0.92575039]
 [ 0.04205607  0.87266355]
 [ 0.04217432  0.84442362]
 [ 0.05631659  0.8196347 ]
 [ 0.08529599  0.78485041]
 [ 0.0821256   0.76570048]
 [ 0.11088608  0.72405063]
 [ 0.10427574  0.71465969]
 [ 0.12905878  0.68475134

In [86]:
def add_label_win1(pc,bins,p_real):
    for loc in range(len(bins)):
        if pc<bins[loc]:
            break
    loc-=1
    return float(p_real[loc,0])

def add_label_win2(pc,bins,p_real):
    for loc in range(len(bins)):
        if pc<bins[loc]:
            break
    loc-=1
    return float(p_real[loc,1])
#udf_add_win1=F.udf(add_label_win1,T.FloatType())
udf_add_win1=F.udf(lambda x: add_label_win1(x,bins,p_real),T.FloatType())
udf_add_win2=F.udf(lambda x: add_label_win2(x,bins,p_real),T.FloatType())
#test=data.select("*",udf_add_win1(data.pc,bins,p_real).alias('label_win1'))
#test=data.withColumn('label_win1',udf_add_win1(data.pc))
data=data.select("*",udf_add_win1(data.pc).alias('label_win1'))
data=data.select("*",udf_add_win2(data.pc).alias('label_win2'))
data=data.drop("scoreA")
data=data.drop("scoreB")
data.show()

+---------+-------+---------+---------+-------+---------+------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+----------+----------+
|avg_win_1|avg_tie|avg_win_2|max_win_1|max_tie|max_win_2|result|                 pc|                 f1|                 f2|                 f3|                 f4|                 f5|label_win1|label_win2|
+---------+-------+---------+---------+-------+---------+------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+----------+----------+
|   2.9944| 3.1944|   2.2256|      3.2|   3.25|     2.29|     2|0.33395671920919046|0.31304783370899075|0.44931703810208484|             0.3125| 0.3076923076923077| 0.4366812227074236| 0.2918065|0.41764542|
|   1.9456| 3.2333|   3.6722|     2.04|    3.3|     4.15|     0| 0.5139802631578947|  0.309281538984938|  0.272316322640379|0.49019607843137253|0.30303030303030304|0.240963

In [88]:
data1=data.select(data["label_win1"],data["pc"],data["f1"],data["f2"],data["f3"],data["f4"],data["f5"])
data2=data.select(data["label_win2"],data["pc"],data["f1"],data["f2"],data["f3"],data["f4"],data["f5"])
data1.show()

+----------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|label_win1|                 pc|                 f1|                 f2|                 f3|                 f4|                 f5|
+----------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
| 0.2918065|0.33395671920919046|0.31304783370899075|0.44931703810208484|             0.3125| 0.3076923076923077| 0.4366812227074236|
| 0.4817307| 0.5139802631578947|  0.309281538984938|  0.272316322640379|0.49019607843137253|0.30303030303030304|0.24096385542168672|
|0.51008594| 0.5398984990821726|0.30664499708687254|0.24910322837783977|                0.5|0.29411764705882354| 0.2222222222222222|
| 0.5839727| 0.6202704379109292|0.29297161105088915|0.18274185885018823| 0.5988023952095809| 0.2801120448179272| 0.1594896331738437|
|0.13869159|0.16700624603360165| 0.2875380987980908| 0.64238453138048

In [90]:
# Import `DenseVector`
from pyspark.ml.linalg import DenseVector

# Define the `input_data` 
input_data1 = data1.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

# Replace `df` with the new DataFrame
df1 = spark.createDataFrame(input_data1, ["label", "features"])
df1.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|0.29180648922920227|[0.33395671920919...|
|0.48173069953918457|[0.51398026315789...|
|  0.510085940361023|[0.53989849908217...|
|  0.583972692489624|[0.62027043791092...|
|0.13869158923625946|[0.16700624603360...|
| 0.5403007864952087|[0.58823529411764...|
| 0.4825265407562256|[0.53699924820105...|
| 0.5527113676071167|[0.61184532550171...|
|  0.419222354888916|[0.45134500812421...|
| 0.2706204354763031|[0.29752164469965...|
| 0.5403007864952087|[0.59259259259259...|
|0.33076611161231995|[0.36330608537693...|
|  0.510085940361023|[0.54457332679845...|
| 0.4825265407562256|[0.53547523427041...|
| 0.3842010796070099|[0.42215467747382...|
| 0.3612334728240967|[0.40139686107654...|
|  0.510085940361023|[0.54383293452251...|
| 0.3842010796070099|[0.42238648363252...|
|0.41165459156036377|[0.44444444444444...|
|0.35324230790138245|[0.39427512518235...|
+----------

In [91]:
# Define the `input_data` 
input_data2 = data1.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

# Replace `df` with the new DataFrame
df2 = spark.createDataFrame(input_data2, ["label", "features"])
df2.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|0.29180648922920227|[0.33395671920919...|
|0.48173069953918457|[0.51398026315789...|
|  0.510085940361023|[0.53989849908217...|
|  0.583972692489624|[0.62027043791092...|
|0.13869158923625946|[0.16700624603360...|
| 0.5403007864952087|[0.58823529411764...|
| 0.4825265407562256|[0.53699924820105...|
| 0.5527113676071167|[0.61184532550171...|
|  0.419222354888916|[0.45134500812421...|
| 0.2706204354763031|[0.29752164469965...|
| 0.5403007864952087|[0.59259259259259...|
|0.33076611161231995|[0.36330608537693...|
|  0.510085940361023|[0.54457332679845...|
| 0.4825265407562256|[0.53547523427041...|
| 0.3842010796070099|[0.42215467747382...|
| 0.3612334728240967|[0.40139686107654...|
|  0.510085940361023|[0.54383293452251...|
| 0.3842010796070099|[0.42238648363252...|
|0.41165459156036377|[0.44444444444444...|
|0.35324230790138245|[0.39427512518235...|
+----------

In [109]:
train1,test1=df1.randomSplit([.8,.2],seed=1234)
train1.show()
test1.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[0.01698923562031...|
|  0.0|[0.01704181379432...|
|  0.0|[0.01705236269011...|
|  0.0|[0.01720841629224...|
|  0.0|[0.01753017381167...|
|  0.0|[0.01777291194943...|
|  0.0|[0.01798354146285...|
|  0.0|[0.01879328328055...|
|  0.0|[0.01906781276933...|
|  0.0|[0.01909559445540...|
|  0.0|[0.01949078378288...|
|  0.0|[0.01973682912050...|
|  0.0|[0.01997040386147...|
|  0.0|[0.02039754821470...|
|  0.0|[0.02046035805626...|
|  0.0|[0.02075394947658...|
|  0.0|[0.02077546500684...|
|  0.0|[0.02087682672233...|
|  0.0|[0.02216046838365...|
|  0.0|[0.02230778497079...|
+-----+--------------------+
only showing top 20 rows

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[0.01647804794452...|
|  0.0|[0.01680915224721...|
|  0.0|[0.01691795468695...|
|  0.0|[0.01913450793699...|
|  0.0|[0.02332932846528...|
|  0.0|[0.02335695500049...|
|  0.0|[0.0249599

In [112]:
from pyspark.ml.regression import LinearRegression
#lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr = LinearRegression()

# Fit the model
lrModel = lr.fit(train1)

# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))


Coefficients: [0.995483379561,-0.041943936156,0.0208736708151,0.00556878229994,-0.00523700416862,-0.00754622318807]
Intercept: -0.0273565604229


In [ ]:
test=df1.select(df1["features"])
#test.show()

In [114]:
predicted = lrModel.transform(test)

In [115]:
predicted.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|[0.33395671920919...| 0.2981738052387201|
|[0.51398026315789...| 0.4763384623756539|
|[0.53989849908217...| 0.5020083799910259|
|[0.62027043791092...| 0.5823025700717768|
|[0.16700624603360...|0.13492647985937922|
|[0.58823529411764...| 0.5503857812497094|
|[0.53699924820105...|0.49921216246875266|
|[0.61184532550171...| 0.5738977204823215|
|[0.45134500812421...|0.41412657465419983|
|[0.29752164469965...| 0.2622720466701907|
|[0.59259259259259...| 0.5550554002436286|
|[0.36330608537693...|0.32749263192503236|
|[0.54457332679845...| 0.5067067426279065|
|[0.53547523427041...|0.49786857626439607|
|[0.42215467747382...|0.38573217323217046|
|[0.40139686107654...| 0.3649627989115332|
|[0.54383293452251...| 0.5064179580208961|
|[0.42238648363252...|0.38594107031802866|
|[0.44444444444444...|0.40772792783752665|
|[0.39427512518235...| 0.3580741261793817|
+----------

In [116]:
df1.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|0.29180648922920227|[0.33395671920919...|
|0.48173069953918457|[0.51398026315789...|
|  0.510085940361023|[0.53989849908217...|
|  0.583972692489624|[0.62027043791092...|
|0.13869158923625946|[0.16700624603360...|
| 0.5403007864952087|[0.58823529411764...|
| 0.4825265407562256|[0.53699924820105...|
| 0.5527113676071167|[0.61184532550171...|
|  0.419222354888916|[0.45134500812421...|
| 0.2706204354763031|[0.29752164469965...|
| 0.5403007864952087|[0.59259259259259...|
|0.33076611161231995|[0.36330608537693...|
|  0.510085940361023|[0.54457332679845...|
| 0.4825265407562256|[0.53547523427041...|
| 0.3842010796070099|[0.42215467747382...|
| 0.3612334728240967|[0.40139686107654...|
|  0.510085940361023|[0.54383293452251...|
| 0.3842010796070099|[0.42238648363252...|
|0.41165459156036377|[0.44444444444444...|
|0.35324230790138245|[0.39427512518235...|
+----------